# Regentfund Allocation Strategy

**Proportional Linear Momentum Strategy for Fixed Income Allocation**

This notebook implements two variants:
- **Strategy 1**: Base momentum strategy with cross-asset regime filter and mom240 exit condition
- **Strategy 2**: Strategy 1 + Drawdown Differential Override (asset-level)

---

## Assets
| Code | Description | Benchmark |
|------|-------------|----------|
| H5A4 | High Yield | SPY |
| C5A4 | Corporate (IG) | LQD |
| G502 | Government (Long) | TLT |

In [4]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Imports loaded")

Imports loaded


In [5]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Portfolio Constraints
CONSTRAINTS = {
    'Cash': {'min': 0.05, 'max': 0.10},
    'H5A4': {'min': 0.00, 'max': 0.12},  # HY capped at 12%
    'C5A4': {'min': 0.00, 'max': 0.95},
    'G502': {'min': 0.00, 'max': 0.95}
}

# Strategy 1: Signal Generation Thresholds (per asset)
SIGNAL_THRESHOLDS = {
    'H5A4': -0.03,   # Exit threshold vs SPY
    'C5A4': -0.013,  # Exit threshold vs LQD
    'G502': -0.01    # Exit threshold vs TLT
}

# Defensive Allocation (when no signals active)
DEFENSIVE_ALLOCATION = {'Cash': 0.10, 'H5A4': 0.00, 'C5A4': 0.00, 'G502': 0.90}

print("Configuration loaded")
print(f"\nSignal Thresholds:")
for asset, thresh in SIGNAL_THRESHOLDS.items():
    print(f"  {asset}: {thresh}")
print(f"\nDefensive Allocation: {DEFENSIVE_ALLOCATION}")
print(f"\nNote: Strategy 2 DD override thresholds are set in individual asset analysis files")

Configuration loaded

Signal Thresholds:
  H5A4: -0.03
  C5A4: -0.013
  G502: -0.01

Defensive Allocation: {'Cash': 0.1, 'H5A4': 0.0, 'C5A4': 0.0, 'G502': 0.9}

Note: Strategy 2 DD override thresholds are set in individual asset analysis files


In [7]:
# =============================================================================
# DATA LOADING
# =============================================================================

# Load ETF data for HYG BMOM calculation
etf_data = pd.read_csv('Fixed_Income_Data.csv', parse_dates=['Date'], index_col='Date')
etf_data.index = pd.to_datetime(etf_data.index)

# Calculate HYG BMOM (weighted momentum)
hyg = etf_data['HYG']
HYG_BMOM = (0.15 * hyg.pct_change(20) + 0.35 * hyg.pct_change(60) + 
            0.35 * hyg.pct_change(120) + 0.15 * hyg.pct_change(240))

# Load asset-specific data from Excel files
h5a4_data = pd.read_excel('High Yield/Taiya_Shiva_Bond_TAA_Stats_Visual.xlsx', 
                          sheet_name='Strategy_Data', index_col='Date', parse_dates=True)
c5a4_data = pd.read_excel('Corporate/LQD_C5A4_Analysis.xlsx', 
                          sheet_name='Strategy_Data', index_col='Date', parse_dates=True)
g502_data = pd.read_excel('Government/TLT_G502_Analysis.xlsx', 
                          sheet_name='Strategy_Data', index_col='Date', parse_dates=True)

# Consolidate into single DataFrame
data = pd.DataFrame()

# Benchmark BMOMs
data['SPY_BMOM'] = h5a4_data['SPY_BMOM']
data['TLT_BMOM'] = g502_data['TLT_BMOM']
data['LQD_BMOM'] = c5a4_data['LQD_BMOM']
data['HYG_BMOM'] = HYG_BMOM

# Asset BMOMs
data['H5A4_BMOM'] = h5a4_data['H5A4_BMOM']
data['C5A4_BMOM'] = c5a4_data['C5A4_BMOM']
data['G502_BMOM'] = g502_data['G502_BMOM']

# Asset Prices
data['H5A4_Price'] = h5a4_data['H5A4_Price']
data['C5A4_Price'] = c5a4_data['C5A4_Price']
data['G502_Price'] = g502_data['G502_Price']

# Mom240 (240-day momentum for exit filter)
data['H5A4_Mom240'] = h5a4_data['H5A4_Mom240']
data['C5A4_Mom240'] = c5a4_data['C5A4_Mom240']
data['G502_Mom240'] = g502_data['G502_Mom240']

# Sort and clean
data = data.sort_index()
data.index = pd.to_datetime(data.index)
data.index.name = 'Date'

# Filter to first common date (no NaN in core columns)
core_cols = ['SPY_BMOM', 'TLT_BMOM', 'LQD_BMOM', 'HYG_BMOM', 
             'H5A4_BMOM', 'C5A4_BMOM', 'G502_BMOM',
             'H5A4_Price', 'C5A4_Price', 'G502_Price', 
             'H5A4_Mom240', 'C5A4_Mom240', 'G502_Mom240']
first_date = data[core_cols].dropna().index.min()
data = data.loc[first_date:].copy()

# Calculate daily returns
data['H5A4_Return'] = data['H5A4_Price'].pct_change()
data['C5A4_Return'] = data['C5A4_Price'].pct_change()
data['G502_Return'] = data['G502_Price'].pct_change()

print(f"Data loaded successfully")
print(f"  Shape: {data.shape}")
print(f"  Date range: {data.index.min().date()} to {data.index.max().date()}")
print(f"  Trading days: {len(data):,}")

ValueError: Missing column provided to 'parse_dates': 'Date'

---
## Strategy 1: Proportional Linear

**Signal Logic:**
- Entry (signal=1): Asset BMOM > 0
- Exit (signal=0): (Benchmark OR Asset crosses below threshold while BOTH below threshold) AND (Mom240 < 0)
- Hold: Keep previous signal

In [ ]:
# =============================================================================
# STRATEGY 1: SIGNAL GENERATION
# =============================================================================

def generate_strategy1_signal(asset_bmom, benchmark_bmom, asset_mom240, threshold):
    """
    Generate Strategy 1 signals for a single asset.
    
    Entry (signal=1): Asset BMOM > 0
    Exit (signal=0):  (Benchmark OR Asset crosses below threshold 
                       while BOTH below threshold) AND (Mom240 < 0)
    Hold: Keep previous signal
    """
    signals = pd.Series(0, index=asset_bmom.index)
    
    for i in range(1, len(asset_bmom)):
        # Current values
        bench_curr = benchmark_bmom.iloc[i]
        asset_curr = asset_bmom.iloc[i]
        mom240_curr = asset_mom240.iloc[i]
        
        # Previous values
        bench_prev = benchmark_bmom.iloc[i-1]
        asset_prev = asset_bmom.iloc[i-1]
        
        # Exit Condition 1: Benchmark crosses below threshold while both below threshold
        cond1 = (bench_prev > threshold) and (bench_curr < threshold) and (asset_curr < threshold)
        
        # Exit Condition 2: Asset crosses below threshold while both below threshold
        cond2 = (asset_prev > threshold) and (bench_curr < threshold) and (asset_curr < threshold)
        
        # Mom240 must be negative for exit
        mom240_negative = mom240_curr < 0
        
        if (cond1 or cond2) and mom240_negative:
            # Exit
            signals.iloc[i] = 0
        elif asset_curr > 0:
            # Entry
            signals.iloc[i] = 1
        else:
            # Hold
            signals.iloc[i] = signals.iloc[i-1]
    
    return signals

# Generate Strategy 1 signals for each asset
signals_strat1 = pd.DataFrame(index=data.index)

signals_strat1['H5A4'] = generate_strategy1_signal(
    data['H5A4_BMOM'], data['SPY_BMOM'], data['H5A4_Mom240'], SIGNAL_THRESHOLDS['H5A4'])

signals_strat1['C5A4'] = generate_strategy1_signal(
    data['C5A4_BMOM'], data['LQD_BMOM'], data['C5A4_Mom240'], SIGNAL_THRESHOLDS['C5A4'])

signals_strat1['G502'] = generate_strategy1_signal(
    data['G502_BMOM'], data['TLT_BMOM'], data['G502_Mom240'], SIGNAL_THRESHOLDS['G502'])

# Summary
print("Strategy 1 Signal Distribution:")
print("=" * 60)
print(f"{'Asset':<8} {'Benchmark':<12} {'Threshold':<12} {'Long %':<10} {'Flat %':<10}")
print("-" * 60)
for asset, bench in [('H5A4', 'SPY'), ('C5A4', 'LQD'), ('G502', 'TLT')]:
    long_pct = signals_strat1[asset].mean() * 100
    thresh = SIGNAL_THRESHOLDS[asset]
    print(f"{asset:<8} {bench:<12} {thresh:<12.3f} {long_pct:<10.1f} {100-long_pct:<10.1f}")
print("=" * 60)

In [ ]:
# =============================================================================
# STRATEGY 1: PROPORTIONAL LINEAR WEIGHTS
# =============================================================================

def calc_proportional_weights(signals_df, bmom_data):
    """
    Calculate proportional weights based on momentum scores.
    
    Score = max(0, BMOM) * signal
    Weight = (score / total_score) * 95%
    HY (H5A4) capped at 12%
    Cash minimum 5%
    Defensive: 10% cash + 90% G502 when no signals
    """
    weights = pd.DataFrame(index=signals_df.index, columns=['Cash', 'H5A4', 'C5A4', 'G502'])
    
    for i in range(len(signals_df)):
        # Calculate scores (excess BMOM * signal)
        scores = {}
        for asset in ['H5A4', 'C5A4', 'G502']:
            excess_bmom = max(0, bmom_data[f'{asset}_BMOM'].iloc[i])
            scores[asset] = excess_bmom * signals_df[asset].iloc[i]
        
        total_score = sum(scores.values())
        
        if total_score == 0:
            # Defensive allocation
            weights.iloc[i] = [DEFENSIVE_ALLOCATION['Cash'], 
                               DEFENSIVE_ALLOCATION['H5A4'],
                               DEFENSIVE_ALLOCATION['C5A4'], 
                               DEFENSIVE_ALLOCATION['G502']]
        else:
            cash = CONSTRAINTS['Cash']['min']  # 5%
            available = 1 - cash  # 95%
            
            # Proportional allocation
            raw = {asset: (scores[asset] / total_score) * available for asset in ['H5A4', 'C5A4', 'G502']}
            
            # Apply HY cap (12%)
            if raw['H5A4'] > CONSTRAINTS['H5A4']['max']:
                excess = raw['H5A4'] - CONSTRAINTS['H5A4']['max']
                raw['H5A4'] = CONSTRAINTS['H5A4']['max']
                
                # Redistribute excess to other active assets
                if scores['C5A4'] > 0:
                    raw['C5A4'] += excess
                elif scores['G502'] > 0:
                    raw['G502'] += excess
                else:
                    cash += excess
            
            weights.iloc[i] = [cash, raw['H5A4'], raw['C5A4'], raw['G502']]
    
    return weights.astype(float)

# Calculate Strategy 1 weights
weights_strat1 = calc_proportional_weights(signals_strat1, data)

# Calculate Strategy 1 returns (using previous day's weights)
returns_strat1 = (weights_strat1['H5A4'].shift(1) * data['H5A4_Return'] +
                  weights_strat1['C5A4'].shift(1) * data['C5A4_Return'] +
                  weights_strat1['G502'].shift(1) * data['G502_Return']).fillna(0)

equity_strat1 = (1 + returns_strat1).cumprod()

print(f"Strategy 1 - Final Equity: {equity_strat1.iloc[-1]:.4f}")
print(f"Strategy 1 - Total Return: {(equity_strat1.iloc[-1] - 1) * 100:.2f}%")

---
## Strategy 2: Proportional Linear + DD Override

**Strategy 2 uses pre-computed signals from individual asset analysis files.**

These signals incorporate:
- Strategy 1 base logic (momentum with cross-asset regime filter and mom240 exit)
- DD Override: Forces position to long when strategy underperformance vs buy-and-hold exceeds threshold
- State-based exit conditions to prevent premature exits during override periods

The DD override logic is computed separately for each asset in their respective analysis files.

In [ ]:
# =============================================================================
# BENCHMARK (for DD calculation)
# =============================================================================

# Equal weight benchmark
returns_benchmark = (data['H5A4_Return'] + data['C5A4_Return'] + data['G502_Return']) / 3
equity_benchmark = (1 + returns_benchmark.fillna(0)).cumprod()

print(f"Benchmark (Equal Weight) - Final Equity: {equity_benchmark.iloc[-1]:.4f}")

In [ ]:
# =============================================================================
# STRATEGY 2: LOAD PRE-COMPUTED SIGNALS WITH DD OVERRIDE
# =============================================================================

# Load Strategy 2 signals from individual asset analysis files
# These include the DD override logic with state-based exit conditions

# Load H5A4 Strategy 2 signals - from CSV
h5a4_strat2_data = pd.read_csv('High Yield/H5A4_Strat2_Threshold_Analysis.csv', 
                                index_col='Date', parse_dates=True)

# Load C5A4 Strategy 2 signals
c5a4_strat2_data = pd.read_excel('Corporate/LQD_C5A4_Analysis.xlsx',
                                  sheet_name='Strategy_Data', index_col='Date', parse_dates=True)

# Load G502 Strategy 2 signals  
g502_strat2_data = pd.read_excel('Government/TLT_G502_Analysis.xlsx',
                                  sheet_name='Strategy_Data', index_col='Date', parse_dates=True)

# Create Strategy 2 signals DataFrame
signals_strat2 = pd.DataFrame(index=data.index)

# For H5A4, check which threshold column to use (we'll use 05 = 5% threshold as default)
# Available columns in H5A4 file: Strat2_02_Signal, Strat2_03_Signal, etc.
if 'Strat2_05_Signal' in h5a4_strat2_data.columns:
    signals_strat2['H5A4'] = h5a4_strat2_data['Strat2_05_Signal'].reindex(data.index)
elif 'Strat2_Signal' in h5a4_strat2_data.columns:
    signals_strat2['H5A4'] = h5a4_strat2_data['Strat2_Signal'].reindex(data.index)
else:
    # Use Strat2_03_Signal (3% threshold) if available
    signals_strat2['H5A4'] = h5a4_strat2_data['Strat2_03_Signal'].reindex(data.index)

signals_strat2['C5A4'] = c5a4_strat2_data['Strat2_Signal'].reindex(data.index)
signals_strat2['G502'] = g502_strat2_data['Strat2_Signal'].reindex(data.index)

# Fill any NaN values with 0 (flat position)
signals_strat2 = signals_strat2.fillna(0).astype(int)

# Summary
print("Strategy 2 Signal Distribution (Strat1 + DD Override):")
print("=" * 60)
print(f"{'Asset':<8} {'Benchmark':<12} {'Threshold':<12} {'Long %':<10} {'Flat %':<10}")
print("-" * 60)
for asset, bench in [('H5A4', 'SPY'), ('C5A4', 'LQD'), ('G502', 'TLT')]:
    long_pct = signals_strat2[asset].mean() * 100
    thresh = SIGNAL_THRESHOLDS[asset]
    print(f"{asset:<8} {bench:<12} {thresh:<12.3f} {long_pct:<10.1f} {100-long_pct:<10.1f}")
print("=" * 60)

# Compare to Strategy 1
print("\nSignal Changes (Strat1 -> Strat2):")
print("=" * 60)
for asset in ['H5A4', 'C5A4', 'G502']:
    s1_long = signals_strat1[asset].mean() * 100
    s2_long = signals_strat2[asset].mean() * 100
    change = s2_long - s1_long
    print(f"{asset}: {s1_long:.1f}% -> {s2_long:.1f}% (change: {change:+.1f} percentage points)")
print("=" * 60)

In [ ]:
# =============================================================================
# STRATEGY 2: PROPORTIONAL LINEAR WEIGHTS
# =============================================================================

# Calculate Strategy 2 weights (using same weight function)
weights_strat2 = calc_proportional_weights(signals_strat2, data)

# Calculate Strategy 2 returns
returns_strat2 = (weights_strat2['H5A4'].shift(1) * data['H5A4_Return'] +
                  weights_strat2['C5A4'].shift(1) * data['C5A4_Return'] +
                  weights_strat2['G502'].shift(1) * data['G502_Return']).fillna(0)

equity_strat2 = (1 + returns_strat2).cumprod()

print(f"Strategy 2 - Final Equity: {equity_strat2.iloc[-1]:.4f}")
print(f"Strategy 2 - Total Return: {(equity_strat2.iloc[-1] - 1) * 100:.2f}%")

---
## Performance Comparison

In [ ]:
# =============================================================================
# PERFORMANCE METRICS
# =============================================================================

def calc_metrics(returns, name):
    """Calculate comprehensive performance metrics."""
    n = len(returns)
    total = (1 + returns).prod() - 1
    ann_ret = (1 + total) ** (252 / n) - 1
    ann_vol = returns.std() * np.sqrt(252)
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
    
    equity = (1 + returns).cumprod()
    max_dd = ((equity - equity.expanding().max()) / equity.expanding().max()).min()
    
    # Sortino ratio
    downside_returns = returns[returns < 0]
    downside_vol = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino = ann_ret / downside_vol if downside_vol > 0 else 0
    
    return {
        'Strategy': name,
        'Ann_Return': ann_ret,
        'Volatility': ann_vol,
        'Sharpe': sharpe,
        'Sortino': sortino,
        'Max_DD': max_dd,
        'Total_Return': total
    }

# Calculate metrics for all strategies
metrics = [
    calc_metrics(returns_strat1, 'Strategy 1 (Base)'),
    calc_metrics(returns_strat2, 'Strategy 2 (DD Override)'),
    calc_metrics(returns_benchmark.fillna(0), 'Benchmark (EW)')
]
metrics_df = pd.DataFrame(metrics)

# Display
print("\nPERFORMANCE COMPARISON")
print("=" * 100)
print(f"{'Strategy':<25} {'Ann Return':>12} {'Volatility':>12} {'Sharpe':>10} {'Sortino':>10} {'Max DD':>12} {'Total':>12}")
print("-" * 100)
for _, r in metrics_df.iterrows():
    print(f"{r['Strategy']:<25} {r['Ann_Return']:>11.2%} {r['Volatility']:>11.2%} "
          f"{r['Sharpe']:>9.2f} {r['Sortino']:>9.2f} {r['Max_DD']:>11.2%} {r['Total_Return']:>11.2%}")
print("=" * 100)

In [ ]:
# =============================================================================
# EQUITY CURVE VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Panel 1: Equity Curves
ax = axes[0]
ax.plot(equity_strat1.index, equity_strat1, label='Strategy 1 (Base)', linewidth=2, alpha=0.9)
ax.plot(equity_strat2.index, equity_strat2, label='Strategy 2 (DD Override)', linewidth=2, alpha=0.9)
ax.plot(equity_benchmark.index, equity_benchmark, label='Benchmark (EW)', linewidth=2, alpha=0.7, linestyle='--')

ax.set_title('Equity Curves: Strategy 1 vs Strategy 2 vs Benchmark', fontsize=14, fontweight='bold')
ax.set_ylabel('Equity', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(data.index.min(), data.index.max())

# Panel 2: Signal Comparison (Strategy 1 vs Strategy 2)
ax = axes[1]
colors = {'H5A4': 'red', 'C5A4': 'blue', 'G502': 'green'}

for i, asset in enumerate(['H5A4', 'C5A4', 'G502']):
    # Plot where signals differ (Strat2 = 1, Strat1 = 0)
    override_active = ((signals_strat2[asset] == 1) & (signals_strat1[asset] == 0)).astype(int)
    
    ax.fill_between(data.index, 
                    override_active * (i + 1),
                    i,
                    where=(override_active > 0),
                    color=colors[asset], alpha=0.5, label=f'{asset} Override Active')

ax.set_title('DD Override Active Zones (Strat2 Long when Strat1 Flat)', fontsize=14, fontweight='bold')
ax.set_ylabel('Asset', fontsize=12)
ax.set_yticks([0.5, 1.5, 2.5])
ax.set_yticklabels(['H5A4', 'C5A4', 'G502'])
ax.set_xlabel('Date', fontsize=12)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3, axis='x')
ax.set_xlim(data.index.min(), data.index.max())

plt.tight_layout()
plt.show()

---
## Turnover Analysis

In [ ]:
# =============================================================================
# TURNOVER ANALYSIS
# =============================================================================

def analyze_turnover(weights, strategy_name):
    """Calculate turnover statistics for a strategy."""
    weight_changes = weights.diff().abs()
    daily_turnover = weight_changes.sum(axis=1)
    
    total_days = len(daily_turnover)
    years = total_days / 252
    
    stats = {
        'strategy': strategy_name,
        'total_turnover': daily_turnover.sum(),
        'annualized_turnover': daily_turnover.sum() / years,
        'mean_daily': daily_turnover.mean(),
        'max_daily': daily_turnover.max(),
        'days_gt_1pct': (daily_turnover > 0.01).sum(),
        'trading_days': (daily_turnover > 0.001).sum()
    }
    
    return stats, daily_turnover

# Analyze both strategies
stats_s1, turnover_s1 = analyze_turnover(weights_strat1, 'Strategy 1')
stats_s2, turnover_s2 = analyze_turnover(weights_strat2, 'Strategy 2')

print("\nTURNOVER COMPARISON")
print("=" * 80)
print(f"{'Metric':<30} {'Strategy 1':>20} {'Strategy 2':>20}")
print("-" * 80)
print(f"{'Total Turnover':<30} {stats_s1['total_turnover']:>19.2f}x {stats_s2['total_turnover']:>19.2f}x")
print(f"{'Annualized Turnover':<30} {stats_s1['annualized_turnover']:>19.2f}x {stats_s2['annualized_turnover']:>19.2f}x")
print(f"{'Mean Daily Turnover':<30} {stats_s1['mean_daily']*100:>18.2f}% {stats_s2['mean_daily']*100:>18.2f}%")
print(f"{'Max Daily Turnover':<30} {stats_s1['max_daily']*100:>18.2f}% {stats_s2['max_daily']*100:>18.2f}%")
print(f"{'Days with >1% Turnover':<30} {stats_s1['days_gt_1pct']:>20} {stats_s2['days_gt_1pct']:>20}")
print(f"{'Total Trading Days':<30} {stats_s1['trading_days']:>20} {stats_s2['trading_days']:>20}")
print("=" * 80)

In [ ]:
# =============================================================================
# TRADE-BY-TRADE LOG (Strategy 1)
# =============================================================================

TRADE_THRESHOLD = 0.01  # 1% turnover = significant trade

def generate_trade_log(weights, signals, daily_turnover, data, strategy_name):
    """Generate detailed trade log."""
    trade_days = daily_turnover[daily_turnover > TRADE_THRESHOLD]
    
    print(f"\n{'='*100}")
    print(f"TRADE LOG - {strategy_name}")
    print(f"{'='*100}")
    print(f"Trade Definition: Daily turnover > {TRADE_THRESHOLD*100:.1f}%")
    print(f"Total Trades: {len(trade_days)}")
    
    trade_log = []
    cumulative_to = 0
    
    for i, (date, turnover) in enumerate(trade_days.items()):
        cumulative_to += turnover
        date_idx = weights.index.get_loc(date)
        prev_date = weights.index[date_idx - 1] if date_idx > 0 else None
        
        curr_weights = weights.loc[date]
        prev_weights = weights.loc[prev_date] if prev_date else pd.Series({'Cash': 0.1, 'H5A4': 0, 'C5A4': 0, 'G502': 0.9})
        
        
        curr_signals = {asset: signals.loc[date, asset] for asset in ['H5A4', 'C5A4', 'G502']}
        
        trade_log.append({
            'Trade #': i + 1,
            'Date': date,
            'Turnover': turnover,
            'Cumulative': cumulative_to,
            'Cash': curr_weights['Cash'],
            'H5A4': curr_weights['H5A4'],
            'C5A4': curr_weights['C5A4'],
            'G502': curr_weights['G502'],
            'Signals': f"H:{int(curr_signals['H5A4'])} C:{int(curr_signals['C5A4'])} G:{int(curr_signals['G502'])}"
        })
    
    # Print trade log
    print(f"\n{'#':<4} {'Date':<12} {'Turnover':>10} {'Cumul':>8} | {'Cash':>8} {'H5A4':>8} {'C5A4':>8} {'G502':>8} | {'Signals':<14}")
    print("-" * 100)
    
    for t in trade_log:
        print(f"{t['Trade #']:<4} {t['Date'].strftime('%Y-%m-%d'):<12} "
              f"{t['Turnover']*100:>9.2f}% {t['Cumulative']:>7.2f}x | "
              f"{t['Cash']*100:>7.1f}% {t['H5A4']*100:>7.1f}% {t['C5A4']*100:>7.1f}% {t['G502']*100:>7.1f}% | "
              f"{t['Signals']:<14}")
    
    print("-" * 100)
    print(f"TOTAL: {len(trade_log)} trades, {cumulative_to:.2f}x cumulative turnover")
    
    return pd.DataFrame(trade_log)

# Generate trade logs
trade_log_s1 = generate_trade_log(weights_strat1, signals_strat1, turnover_s1, data, 'Strategy 1')

In [ ]:
# =============================================================================
# MONTHLY TRADING FREQUENCY
# =============================================================================

def analyze_monthly_frequency(trade_log_df, daily_turnover, strategy_name):
    """Analyze monthly trading frequency."""
    if len(trade_log_df) == 0:
        print(f"No trades to analyze for {strategy_name}")
        return None
    
    trade_log_df['YearMonth'] = trade_log_df['Date'].dt.to_period('M')
    monthly_trades = trade_log_df.groupby('YearMonth').agg({
        'Trade #': 'count',
        'Turnover': 'sum'
    }).rename(columns={'Trade #': 'Num_Trades', 'Turnover': 'Total_Turnover'})
    
    # All trading days (>0.1% turnover)
    all_trading = daily_turnover[daily_turnover > 0.001]
    all_trading_monthly = all_trading.groupby(all_trading.index.to_period('M')).count()
    monthly_trades['Trading_Days'] = all_trading_monthly
    monthly_trades = monthly_trades.fillna(0)
    
    print(f"\n{'='*80}")
    print(f"MONTHLY TRADING FREQUENCY - {strategy_name}")
    print(f"{'='*80}")
    print(f"\n{'Month':<10} {'Trades':>8} {'Trading Days':>14} {'Total Turnover':>16}")
    print("-" * 60)
    
    for month in monthly_trades.index:
        num_trades = int(monthly_trades.loc[month, 'Num_Trades'])
        trading_days = int(monthly_trades.loc[month, 'Trading_Days'])
        total_to = monthly_trades.loc[month, 'Total_Turnover']
        print(f"{str(month):<10} {num_trades:>8} {trading_days:>14} {total_to*100:>15.2f}%")
    
    print("-" * 60)
    print(f"\nSUMMARY:")
    print(f"  Average Trades per Month:       {monthly_trades['Num_Trades'].mean():.2f}")
    print(f"  Max Trades in a Month:          {monthly_trades['Num_Trades'].max():.0f}")
    print(f"  Average Trading Days per Month: {monthly_trades['Trading_Days'].mean():.2f}")
    print(f"  Max Trading Days in a Month:    {monthly_trades['Trading_Days'].max():.0f}")
    print(f"  Average Turnover per Month:     {monthly_trades['Total_Turnover'].mean()*100:.2f}%")
    print(f"  Max Turnover in a Month:        {monthly_trades['Total_Turnover'].max()*100:.2f}%")
    
    return monthly_trades

# Analyze monthly frequency for Strategy 1
monthly_s1 = analyze_monthly_frequency(trade_log_s1, turnover_s1, 'Strategy 1')

In [ ]:
# =============================================================================
# TURNOVER VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Turnover Analysis - Strategy 1 vs Strategy 2', fontsize=14, fontweight='bold')

# Panel 1: Daily Turnover Comparison
ax = axes[0, 0]
ax.plot(turnover_s1.index, turnover_s1 * 100, alpha=0.7, label='Strategy 1', linewidth=1)
ax.plot(turnover_s2.index, turnover_s2 * 100, alpha=0.7, label='Strategy 2', linewidth=1)
ax.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='1% threshold')
ax.set_title('Daily Turnover', fontweight='bold')
ax.set_ylabel('Turnover (%)')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Cumulative Turnover
ax = axes[0, 1]
ax.plot(turnover_s1.index, turnover_s1.cumsum(), label='Strategy 1', linewidth=2)
ax.plot(turnover_s2.index, turnover_s2.cumsum(), label='Strategy 2', linewidth=2)
ax.set_title('Cumulative Turnover', fontweight='bold')
ax.set_ylabel('Cumulative Turnover (x)')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Turnover Distribution
ax = axes[1, 0]
ax.hist(turnover_s1[turnover_s1 > 0] * 100, bins=50, alpha=0.6, label='Strategy 1', edgecolor='black')
ax.hist(turnover_s2[turnover_s2 > 0] * 100, bins=50, alpha=0.6, label='Strategy 2', edgecolor='black')
ax.set_title('Turnover Distribution (non-zero days)', fontweight='bold')
ax.set_xlabel('Daily Turnover (%)')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 4: Monthly Trades (Strategy 1)
ax = axes[1, 1]
if monthly_s1 is not None:
    monthly_s1['Num_Trades'].plot(kind='bar', ax=ax, color='steelblue', alpha=0.7, edgecolor='black')
    ax.axhline(y=monthly_s1['Num_Trades'].mean(), color='red', linestyle='--', 
               label=f"Avg: {monthly_s1['Num_Trades'].mean():.1f}")
ax.set_title('Monthly Trade Count (Strategy 1)', fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Number of Trades')
ax.tick_params(axis='x', rotation=45)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## Summary

In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================

print("\n" + "=" * 80)
print("REGENTFUND ALLOCATION STRATEGY - SUMMARY")
print("=" * 80)

print(f"\nData Period: {data.index.min().date()} to {data.index.max().date()} ({len(data):,} days)")

print(f"\nStrategy 1 (Base Proportional Linear):")
print(f"  Final Equity:     {equity_strat1.iloc[-1]:.4f}")
print(f"  Total Return:     {(equity_strat1.iloc[-1] - 1) * 100:.2f}%")
print(f"  Sharpe Ratio:     {metrics_df[metrics_df['Strategy']=='Strategy 1 (Base)']['Sharpe'].values[0]:.2f}")
print(f"  Max Drawdown:     {metrics_df[metrics_df['Strategy']=='Strategy 1 (Base)']['Max_DD'].values[0]:.2%}")
print(f"  Ann. Turnover:    {stats_s1['annualized_turnover']:.2f}x")

print(f"\nStrategy 2 (DD Override):")
print(f"  Final Equity:     {equity_strat2.iloc[-1]:.4f}")
print(f"  Total Return:     {(equity_strat2.iloc[-1] - 1) * 100:.2f}%")
print(f"  Sharpe Ratio:     {metrics_df[metrics_df['Strategy']=='Strategy 2 (DD Override)']['Sharpe'].values[0]:.2f}")
print(f"  Max Drawdown:     {metrics_df[metrics_df['Strategy']=='Strategy 2 (DD Override)']['Max_DD'].values[0]:.2%}")
print(f"  Ann. Turnover:    {stats_s2['annualized_turnover']:.2f}x")

# Calculate override days for each asset
override_days = {}
for asset in ['H5A4', 'C5A4', 'G502']:
    override_days[asset] = ((signals_strat2[asset] == 1) & (signals_strat1[asset] == 0)).sum()
print(f"  Override Days:    H5A4={override_days['H5A4']}, C5A4={override_days['C5A4']}, G502={override_days['G502']}")

print(f"\nBenchmark (Equal Weight):")
print(f"  Final Equity:     {equity_benchmark.iloc[-1]:.4f}")
print(f"  Total Return:     {(equity_benchmark.iloc[-1] - 1) * 100:.2f}%")

print("\n" + "=" * 80)